In [ ]:
import numpy as np
import pandas as pd
import os
from matplotlib.pyplot import *
import importlib

import ATARI.utils.hdf5 as h5io

from copy import copy
from ATARI.theory.resonance_statistics import make_res_par_avg

from ATARI.sammy_interface import sammy_classes, sammy_functions, template_creator
from ATARI.ModelData.experimental_model import Experimental_Model
from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.particle import Particle, Neutron, Ta181

from ATARI.PiTFAll import fnorm


In [ ]:
sammypath = ''
assert(sammypath != '')

In [ ]:
%matplotlib widget

In [ ]:
Ta181 = Particle(Z=73, A=181, I=3.5, mass=180.94803, name='Ta181')
Ta_pair = Particle_Pair(isotope = "Ta181",
                        resonance_ladder = pd.DataFrame(),
                        formalism = "XCT",
                        energy_range = [200,250],
                        ac = 0.8127,
                        target=Ta181,
                        projectile=Neutron,
                        l_max = 1
)

# print quant number map up to l_max
Ta_pair.map_quantum_numbers(print_out=True)

# J = 3, <D> = 9.0030 eV, <gn2> = 452.56615 meV, <gg2> = 32.0 meV
# J = 4, <D> = 8.3031 eV, <gn2> = 332.24347 meV, <gg2> = 32.0 meV

# add spin group information for both s-wave resonances
Ta_pair.add_spin_group(Jpi='3.0',
                       J_ID=1,
                       D=9.0030,
                       gn2_avg=452.56615, #46.5,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

Ta_pair.add_spin_group(Jpi='4.0',
                       J_ID=2,
                       D=8.3031,
                       gn2_avg=332.24347, #35.5,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

resonance_ladder = Ta_pair.sample_resonance_ladder()
resonance_ladder

### To calculate a doppler broadenned cross section without any other experimental corrections

1. Build experimental model with reaction, energy grid, and temperature you want. Also, empty the inputs for additional alphanumeric commands and Resolution function
2. Use the template generator to make a template corresponding to the experimental model you just made and inspect it - alternatively, you can use the preloaded dop_2sg.inp or dop_1sg.inp template in the ATARI.sammy_interface.sammy_templates folder
4. Run sammy without bayes 


In [ ]:

theo_exp_model = Experimental_Model(
                                 title = "theo", 
                                 reaction = "elastic", 
                                 energy_range = [200, 250], 
                                 template = None, 
                                 energy_grid = np.linspace(200,250,1000), 
                                 temp = (295.0, 0.0),
                                 sammy_inputs= {"alphanumeric":[],
                                             "ResFunc": None},
                                 )
# theo_exp_model

In [ ]:

rto = sammy_classes.SammyRunTimeOptions(sammypath,
                                        Print        = True,
                                        bayes        = False,
                                        keep_runDIR  = False,
                                        sammy_runDIR = 'sammy_runDIR_1')

template_creator.make_input_template('theo.inp',Ta_pair, theo_exp_model, rto)
theo_exp_model.template= os.path.realpath('theo.inp')

In [ ]:
sammyINP = sammy_classes.SammyInputData(
    Ta_pair,
    resonance_ladder,
    template=theo_exp_model.template,
    experiment=theo_exp_model,
    energy_grid = theo_exp_model.energy_grid
)

sammyOUT = sammy_functions.run_sammy(sammyINP, rto)

In [ ]:
sammyOUT.pw

In [ ]:
figure()
plot(sammyOUT.pw.E, sammyOUT.pw.theo_xs)

In [ ]:
# from ATARI.PiTFAll.fnorm import calc_theo_broad_xs_for_all_reaction
# calc_theo_broad_xs_for_all_reaction('/Users/noahwalton/gitlab/sammy/sammy/build/bin/sammy',
#                                      shell,
#                                         particle_pair, 
#                                         resonance_ladder, 
#                                         energy_range,
#                                         temperature,
#                                         target_thickness,
#                                         template, 
#                                         reactions)